In [22]:
from dateutil import parser
from dateutil import relativedelta

import pandas as pd

In [23]:
df = pd.read_csv("../data/cleaned_reviews.tsv", sep="\t")
df.head()

,rating,vote,verified,reviewTime,reviewerName,reviewText,summary
0,4,2,True,"04 6, 2017",Amazon Customer,helpful,Four Stars
1,5,0,True,"03 14, 2017",Amazon Customer,easy understand prompt service,Five Stars
2,4,0,True,"02 14, 2017",Christopher Burnett,girlfriend use quite often,Four Stars
3,5,0,True,"01 29, 2017",Amazon Customer,arrive described happy,Very happy.
4,5,0,True,"01 4, 2017",Iheartmanatees,love dummy series never fails,Love the Dummies Series


In [24]:
df["reviewTime"] = df["reviewTime"].apply(parser.parse)

In [25]:
# Sorts accendingly
lastest_date = list(df["reviewTime"].sort_values())[-1]
lastest_date

Timestamp('2018-04-27 00:00:00')

In [26]:
def quater_diff(date) -> int:
    diff = relativedelta.relativedelta(lastest_date, date)
    quarters = diff.years * 4
    for month in range(1, diff.months + 1):
        if month % 3 == 0:
            quarters += 1

    return quarters


df["quaterDiff"] = df["reviewTime"].apply(quater_diff)
df["quaterDiff"].head()

0    4
1    4
2    4
3    4
4    5
Name: quaterDiff, dtype: int64

In [27]:
def calc_vote_success(month_diff: int, votes: int) -> float:
    soften_success = 2  # Helps soften the division jump
    return votes / (month_diff + soften_success)


df["voteSuccess"] = df[["quaterDiff", "vote"]].apply(
    lambda x: calc_vote_success(*x), axis=1
)

df.sort_values("voteSuccess").tail()

,rating,vote,verified,reviewTime,reviewerName,reviewText,summary,quaterDiff,voteSuccess
11528,5,53,True,2018-02-03,Hannah,great little item do job connect desktop pc wi...,Great little item,0,26.500000
11525,5,62,True,2018-02-07,Jill B.,adapter excellent never fails work great route...,Works great in a big house!,0,31.000000
20967,1,445,True,2015-08-24,AvidConsumer,amazon really need cycle high rat review every...,Ignore the glowing reviews from 10 years ago -...,10,37.083333
20725,1,243,True,2017-01-28,FoothillsJim,phone 30 minute cancel subscription autorenewe...,"Impossible to cancel, Renews WITHOUT the PROMI...",4,40.500000
19405,3,101,True,2018-03-08,HipShopper,reading review inexpensive 895 price tag thoug...,Difficult to differentiate content from advert...,0,50.500000
